In [47]:
import cyecca
import numpy as np
import casadi as ca
from cyecca.lie.group_so3 import SO3Quat, so3

In [120]:
chi = so3.elem(ca.SX.sym("chi", 3))
wb = so3.elem(ca.SX.sym("wb", 3))
A = -wb.ad()
# B = chi.right_jacobian_inv()


def sym33_to_vector6(m):
    return ca.vertcat(m[0, 0], m[0, 1], m[0, 2], m[1, 1], m[1, 2], m[2, 2])


def vector6_to_sym33(v):
    return ca.vertcat(
        ca.horzcat(v[0], v[1], v[2]),
        ca.horzcat(v[1], v[3], v[4]),
        ca.horzcat(v[2], v[4], v[5]),
    )


P0 = ca.triu2symm(ca.SX.sym("P0", ca.Sparsity.upper(3)))
Q = ca.triu2symm(ca.SX.sym("Q", ca.Sparsity.upper(3)))
dt = ca.SX.sym("dt")

# prediction
P1 = ca.triu(P0 + A @ P0 + P0 @ A.T + Q)

f = ca.Function(
    "f",
    [sym33_to_vector6(P0), sym33_to_vector6(Q), wb.param, dt],
    [sym33_to_vector6(P1)],
    ["P0", "Q0", "wb", "dt"],
    ["P1"],
)

In [128]:
P = np.array([1, 0, 0, 1, 0, 1], dtype=float)
dt = 0.01
wb = np.array([1, 2, 3], dtype=float)
Q = 0.1 * np.array([1, 0, 0, 1, 0, 1], dtype=float)
for i in range(1000):
    P = f(P, Q, wb, dt)
P

DM([101, 0, 0, 101, 0, 101])

In [132]:
X = SO3Quat.elem(ca.SX.sym("X", 4))
Xr = SO3Quat.elem(ca.SX.sym("Xr", 4))